In [5]:
import pandas as pd
import os
import torch
import numpy as np
import pandas as pd
import re
import ast
from tqdm.notebook import tqdm
from kss import split_sentences
from sentence_transformers import SentenceTransformer, util
data_original=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/transformed/model train & test data(0408)/train_data_before_vector(0408).csv")
data_normal=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/normal/일반 민원.csv")
data_abnormal=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/abnormal/보이스 피싱.csv")

In [6]:
datanames = ['original','normal','abnormal']

In [7]:
# 입력 content를 ngram 리스트로 변환
def content2ngram(content, n):
    """
        content: 사건 내용 리스트 
        n: ngram으로 자를 단위
    """
    ngram_list = []     # 전체 텍스트 ngram
    for case in content:
        cut_count = len(case)-n     # ngram으로 자를 횟수
        ngram_case = []             # 사건당 ngram
        
        for i in range(cut_count):
            ngram_group = ''     # 합쳐질 ngram
            for j in range(n):
                if j != 0: ngram_group += ' '
                ngram_group += case[i+j]     # 문장 합쳐서 붙임
            ngram_case.append(ngram_group)
        ngram_list.append(ngram_case)
    return ngram_list

In [8]:
# 데이터 ngram 변환
for dataname in datanames:
    rawdata = globals()[f'data_{dataname}']
    for n in [2, 3, 4]:     # 2gram, 3gram, 4gram 데이터 생성
        globals()[f'data_{dataname}_{n}gram'] = content2ngram(rawdata, n)

In [9]:
class SentenceTagger:
    def __init__(self):
        #self.device = torch.device('cuda')
        self.device=torch.device('cpu')
        self.embedding_model = None

        
    # SentenceTransformer 모델 로드
    def set_model(self):
        self.embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        
    # 전체에서 사건별 임베딩
    def sentence_embedding(self, content):
        content_embedding = []     # 전체 임베딩
        for case in tqdm(content):
            case_embedding = []     # 사건별 임베딩
            for sent in case:
                sent_embedding = self.embedding_model.encode(sent, device=self.device)     # 문장별 임베딩
                case_embedding.append(np.array(sent_embedding, dtype=object))
            content_embedding.append(np.array(case_embedding, dtype=object))
        return np.array(content_embedding, dtype=object)

In [10]:
# 문장 임베딩 객체 생성
sent_tagger = SentenceTagger()
sent_tagger.set_model()

In [11]:
# 데이터 ngram별 임베딩
for dataname in datanames:
    for n in [2, 3, 4]:
        cur_content_ngram = globals()[f'data_{dataname}_{n}gram']
        globals()[f'{dataname}_{n}gram_embedding'] = sent_tagger.sentence_embedding(cur_content_ngram)
        print(f' {dataname} {n}gram embedding finished')

  0%|          | 0/3 [00:00<?, ?it/s]

 original 2gram embedding finished


  0%|          | 0/3 [00:00<?, ?it/s]

 original 3gram embedding finished


  0%|          | 0/3 [00:00<?, ?it/s]

...

 abnormal 2gram embedding finished


  0%|          | 0/3 [00:00<?, ?it/s]

 abnormal 3gram embedding finished


  0%|          | 0/3 [00:00<?, ?it/s]

 abnormal 4gram embedding finished


In [12]:
# 데이터 ngram별 임베딩 저장
for dataname in datanames:
    for n in [2, 3, 4]:
        cur_ngram_embedding = globals()[f'{dataname}_{n}gram_embedding']
        np.save(f'{dataname}_{n}gram_embedding.npy', np.array(cur_ngram_embedding, dtype=object))
        print(f' {dataname} {n}gram embedding saved')

 original 2gram embedding saved
 original 3gram embedding saved
 original 4gram embedding saved
 normal 2gram embedding saved
 normal 3gram embedding saved
 normal 4gram embedding saved
 abnormal 2gram embedding saved
 abnormal 3gram embedding saved
 abnormal 4gram embedding saved
